# Grains, molecules, atoms

```
 +- grains
    +- grain_states [each line has a corresponding grain_XXXXX group]
    +- grain_0001
       +- molecule_states [each line has a corresponding molecule_XXXXX group]
       +- molecule_0001
          +- atom_states [one per line]
       +- molecule_0002
          +- atom_states
    +- grain_0002
       +- molecule_states
       +- molecule_0001
          +- atom_states 
```


In [26]:
# The following will be later probably stored as JSON
# only dicts and lists allowed (though not checked, currently)

grainJson={
    '_schema':'grain',
    'identity':{
        'material':{'dtype':'a','shape':'variable'}
    },
    'properties':{
        'eletrical':{
            'freeElectrons':{'dtype':'l','unit':'none'},
            'freeHoles':{'dtype':'l','unit':'none'},
        },
        'physical':{
            'reorganizationEnergyExternal':{'dtype':'d','unit':'eV'}
        },
        'chemical':{},
    },
    'topology':{
        'parent':{'dtype':'l'},
        'cellSize':{'dtype':'d','shape':[3],'unit':'m'},
    },
    'implementation':{
        'boundaryCondition':{'dtype':'a'}
    },
    'molecules':{'path':'../grain_{ROW}/grain_state','schema':'molecule'}
}

moleculeJson={
    '_schema':'molecule',
    'identity':{
        'chemicalName':{'dtype':'a','shape':'variable'},
        'molecularWeight':{'dtype':'d','unit':'Dalton'},
    },
    'properties':{
        'electrical':{
            'HOMO':{'dtype':'d','unit':'eV'},
            'LUMO':{'dtype':'d','unit':'eV'},
            'siteEnergy':{
                'orbital':{'dtype':'d','unit':'eV'},
                'electrostatic':{'dtype':'d','unit':'eV'},
                'polarization':{'dtype':'d','unit':'eV'},
            },
            'transferIntegrals':{'dtype':'d','shape':'variable'},
            'reorganizationEnergyInternal':{
                'anion':{'dtype':'d','unit':'eV'},
                'cation':{'dtype':'d','unit':'eV'}
            },
        },
        'physical':{
            'polarizability':{
                'neutral':{'dtype':'d','shape':[3,3],'unit':'AA^2 s^4 kg^-1'},
                'anion':{'dtype':'d','shape':[3,3],'unit':'AA^2 s^4 kg^-1'},
                'cation':{'dtype':'d','shape':[3,3],'unit':'AA^2 s^4 kg^-1'},
            }
        },
        'chemical':{},
    },
    'topology':{
        'parent':{'dtype':'l','unit':'none'},
        'centerOfMass':{'dtype':'d','shape':[3],'unit':'AA'},
        'symmetryAxis':{'dtype':'d','shape':[3],'unit':'AA'},
        'structureNeighbors':{'dtype':'l','shape':'variable'},
    },
    'implementation':{
        'forceFieldType':{'dtype':'a','shape':'variable'},
    },
    'atoms':{'path':'../molecule_{ROW}/atom_state','schema':'atom'}
}
atomJson={
    '_schema':'atom',
    'identity':{
        'element':{'dtype':'a2'}, # 2 characters ascii
        'atomicNumber':{'dtype':'l'},
        'atomicMass':{'dtype':'d','unit':'Dalton'},
    },
    'properties':{
        'physical':{
            'partialCharge':{
                'neutral': {'dtype':'d','unit':'e'},
                'anion': {'dtype':'d','unit':'e'},
                'cation': {'dtype':'d','unit':'e'},
            },
            'polarizability':{
                'neutral': {'dtype':'d','unit':'AA^2 s^4 kg^-1'},
                'anion': {'dtype':'d','unit':'AA^2 s^4 kg^-1'},
                'cation': {'dtype':'d','unit':'AA^2 s^4 kg^-1'},
            }
        },
        'topology':{
            'parent':{'dtype':'l'},
            'type':{'dtype':'a','shape':'variable'},
            'name':{'dtype':'a','shape':'variable'},
            'position':{'dtype':'d','shape':[3],'unit':'AA'},
            'velocity':{'dtype':'d','shape':[3],'unit':'AA/ps'},
            'structure':{'dtype':'l','shape':'variable'},
        }
    }
}



* dtype `a`: implies `'shape':'variable'` (`a3`, on the other hand, is fixed-size and does not imply variable length)
* dtype `l`: implies `'unit':'none'` (`none` unit is a shorthand for dimensionless unscaled)
* `'path'` creates iterator, `{ROW}` will expand to the row index of the parent struct
* all items must have `unit` (except of `path`, and cases where it is implied)
* item assignment must include units, and the value must be converted to the declared unit

In [30]:
from dataclasses import dataclass
import astropy.units
import astropy.units as u
astropy.units.add_enabled_units([
    astropy.units.def_unit('e',astropy.constants.si.e),
    astropy.units.def_unit('none',astropy.units.dimensionless_unscaled)
])
import numpy as np
import h5py
from pprint import pprint
from typing import *
  
    
@dataclass
class CookedSchema:
    '''Cooked schema so that it is directly usable by h5py/numpy and DataProxy.
    Unfortunately, h5py discards dtype metadata, so units must be stored separately.
    Currently it is stored in both places (see NumpyDataProxy._getUnit) but later,
    only the seaprate storage will be used.
    '''
    dtype: List
    units: dict
    def merge(self,other):
        self.dtype+=other.dtype
        self.units.update(other.units)

def cookSchema(desc,prefix=''):
    '''
    Transform dictionary-structured data schema into compound cooked data.
    The result is used by NumpyDataProxy.
    '''
    assert isinstance(desc,dict)
    ret=CookedSchema(dtype=[],units={})
    for k,v in desc.items():
        if k[0]=='_':
            if k=='_schema': continue # ignore for now
        name=prefix+('.' if prefix else '')+k
        if 'path' in v: continue # skip
        if 'dtype' in v:
            shape=v['shape'] if 'shape' in v else ()
            if isinstance(shape,list): shape=tuple(shape)
            unit=astropy.units.Unit(v['unit']) if 'unit' in v else None
            dtype=v['dtype']
            if dtype=='a':
                dtype=h5py.string_dtype(encoding='utf-8')
                shape=None
            elif shape=='variable':
                dtype=h5py.vlen_dtype(np.dtype(dtype))
                shape=None
            if isinstance(dtype,str):
                md=({'unit':unit} if unit is not None else {})
                # print(f"Unit of {k} is {unit}")
                dtype=np.dtype((dtype,shape),metadata=md)
            ret.dtype+=[(name,dtype)]
            ret.units[name]=unit
        else:
            ret.merge(makeDtypeList(v,prefix=name))
    return ret

class NumpyDataProxy(object):
    '''
    Class for emulating property-like access to compound data type. Values can be both numpy array
    or h5py dataset (later, only h5py dataset will be allowed — due to nested data which are not 
    handled yet).
    
    With h5py, the dataset can be larger than RAM, as all assignments operate on the storage directly.
    
    Units are fully supported, i.e. assignments to a value with unit *must* be an astropy.units.Quantity
    with compatible units, and values are stored (and returned) using the units specified in the schema.
    '''
    def __init__(self,values,prefix='',row=None,units=None):
        self._values=values
        self._leaves=set(values.dtype.names)
        self._parents=set()
        self._row=row
        self._units=units
        for n in self._leaves:
            while '.' in n:
                n=n.rsplit('.',1)[0]
                self._parents.add(n)
        self._prefix=prefix
        
    def _clone(self,key=None,row=None):
        'Clone the current object, possibly changing prefix and/or row (created nested accessor)'
        if row and self._row: raise IndexError(f'Accessor is already indexed, with row={row}.')
        if self._row: row=self._row
        if key: prefix=self._getFQ(key)
        else: prefix=self._prefix
        return self.__class__(values=self._values,prefix=prefix,row=row,units=self._units)
    def _getFQ(self,key):
        'Fully-qualified name for the key'
        return (f'{self._prefix}.{key}' if self._prefix else key)
    def _getUnit(self,fq):
        '''Find unit used for the fully-qualified name; it can be either stored in dtype metadata
        (numpy-only, somewhat obsolete) or in self._units.
        '''
        field=self._values.dtype.fields[fq][0]
        if field.metadata and 'unit' in field.metadata: return field.metadata['unit']
        if self._units: return self._units.get(fq,None)
        return None
    def _assertValues(self):
        if self._values is None: raise RuntimeError('values not yet initialized, use _allocate first')
    def __getattr__(self,key):
        '''
        Override standard attribute getter to allow property-like access to the compound data.
        Names starting with _ are handled normally (otherwise there would be infinite recursion).
        The value returned is either value (for leaf nodes; including units, if defined) or accessor
        to the nested category.
        '''
        if key.startswith('_'): return object.__getattr__(self,key)
        fq=self._getFQ(key)
        if fq in self._leaves:
            self._assertValues()
            unit=self._getUnit(fq)
            if unit:
                value=self._values[fq]
                if self._row: value=value[row]
                return astropy.units.Quantity(value=value,unit=unit)
            if self._row: return self._values[fq][self._row]
            else: return self._values[fq]
        if fq in self._parents: return self._clone(key=key)
        raise AttributeError(f"'{fq}': not such attribute (prefix='{self._prefix}', '{key}')")
    def __setattr__(self,key,val):
        '''
        Override standard attribute setter to allow modification of data in a property-like manner.
        Names starting with _ are handled specially.
        Unit consistency is checked during assignment.
        If row is not specified, broadcasting will be active (this functionality is provided by both
        h5py and numpy), i.e. the whole column will be set.
        '''
        if key.startswith('_'): return object.__setattr__(self,key,val)
        fq=self._getFQ(key=key)
        if fq not in self._leaves: raise AttributeError(f"'{fq}': not such settable attribute (prefix='{self._prefix}', '{key}')")
        self._assertValues()
        field=self._values.dtype.fields[fq][0]
        unit=self._getUnit(fq)
        # print(f"{fq}: unit is {unit}")
        if unit: val2=(astropy.units.Quantity(val).to(unit)).value
        else: val2=val
        if self._row:
            if isinstance(self._values,h5py.Dataset): self._values[self._row,fq]=val2
            else: self._values[self._row][fq]=val2
        else: self._values[fq]=val2
    def __getitem__(self,row): 
        '''Indexing access.
        Note that the index (row) is only remembered and engaged only when leaf node is read/written.
        Thus e.g. foo[2].bar.baz=2 is equal to foo.bar.baz[2]=2.
        '''
        return self._clone(row=row)
    def __dir__(self):
        '''
        List properties (attributes) available for this accessor.
        '''
        return set([n[len(self._prefix)+1:].split('.',1)[0] for n in self._values.dtype.names if n.startswith(self._prefix+'.')])
        

# test numpy
if 1:
    atom,mol,grain=[NumpyDataProxy(np.zeros((10,),dtype=cookSchema(d).dtype)) for d in (atomJson,moleculeJson,grainJson)]
    # pprint(atom)
    atom.identity.element='C'
    print(atom.identity.element)
    mol.identity.molecularWeight=1*u.yg
    atom[3].properties.topology.position=(1,2,3)*u.nm
    atom.properties.topology.velocity=(24,5,77)*u.m/u.s
    print(mol.identity.molecularWeight)
    print(atom.properties.topology.position)
    print(atom.properties.topology.velocity)
    print(dir(atom.properties))
 
# test h5py
with h5py.File('/tmp/test3.h5','w') as h5:
    schema=cookSchema(atomJson)
    atomDataset=h5.create_dataset('atom',(10,),dtype=schema.dtype)
    atom=NumpyDataProxy(atomDataset,units=schema.units)
    #atom,mol,grain=[DataProxy(np.zeros((10,),dtype=makeDtypeList(d))) for d in (atomJson,moleculeJson,grainJson)]
    pprint(atom)
    atom.identity.element='C'
    print(atom.identity.element)
    mol.identity.molecularWeight=1*u.yg
    atom[1].properties.topology.position=(1,2,3)*u.nm
    atom.properties.topology.velocity=(24,5,77)*u.m/u.s
    print(mol.identity.molecularWeight)
    print(atom.properties.topology.position)
    print(atom.properties.topology.velocity)
    print(dir(atom.properties))
 


[b'C' b'C' b'C' b'C' b'C' b'C' b'C' b'C' b'C' b'C']
[0.60221408 0.60221408 0.60221408 0.60221408 0.60221408 0.60221408
 0.60221408 0.60221408 0.60221408 0.60221408] u
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [10. 20. 30.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]] Angstrom
[[0.24 0.05 0.77]
 [0.24 0.05 0.77]
 [0.24 0.05 0.77]
 [0.24 0.05 0.77]
 [0.24 0.05 0.77]
 [0.24 0.05 0.77]
 [0.24 0.05 0.77]
 [0.24 0.05 0.77]
 [0.24 0.05 0.77]
 [0.24 0.05 0.77]] Angstrom / ps
['physical', 'topology']
[b'C' b'C' b'C' b'C' b'C' b'C' b'C' b'C' b'C' b'C']
[0.60221408 0.60221408 0.60221408 0.60221408 0.60221408 0.60221408
 0.60221408 0.60221408 0.60221408 0.60221408] u
[[ 0.  0.  0.]
 [10. 20. 30.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]] Angstrom
[[0.24 0.05 0.77]
 [0.24 0.05 0.77]
 [0.24 0.05 0.77]
 [0.24 0.05 0.77]
 [0.24 0.05 0.77]
 [0.24 0.05 0.77]
 [0.24 0.05 0.77]
